# 데이터 불러오기

In [ ]:
# soynlp를 통해 자주 등장하는 단어들의 조합을 묶어준 csv파일로, 해당 단어들을 mecab 사용자 정의사전에 적용시킴
user_nnp = pd.read_csv('/content/mnt/Shareddrives/Real_News/찬영/user-nnp.csv')

# train 데이터
train = pd.read_csv('/content/mnt/Shareddrives/Real_News/찬영/news_train.csv')

# test 데이터
test = pd.read_csv('/content/mnt/Shareddrives/Real_News/찬영/news_test.csv')

NameError: ignored

# 시간 측정 시작

In [ ]:
import time
start = time.time()

# Library 불러오기

In [ ]:
import pandas as pd   # pandas 불러오기


# 먼저 mecab 설치하기
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git


Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 75 (delta 33), reused 21 (delta 5), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [ ]:
cd Mecab-ko-for-Google-Colab

/content/Mecab-ko-for-Google-Colab


In [ ]:

! bash install_mecab-ko_on_colab190912.sh

from konlpy.tag import Mecab  # mecab 불러오기

Installing konlpy.....
     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 460kB 51.2MB/s 
     |████████████████████████████████| 92kB 11.8MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-01-08 15:03:05--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::6b17:d1f5, 2406:da00:ff00::22c5:2ef4, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downlo

In [ ]:
cd /content/mecab-ko-dic-2.1.1-20180720

In [ ]:

# 사용자 정의 데이터 저장
user_nnp.to_csv('/content/mecab-ko-dic-2.1.1-20180720/user-nnp.csv',index=False)

# 이전 컴파일 삭제시키고 새로운 데이터 불러오기
!make clean

!make install

# 이제 mecab함수를 불러와서 토큰화 진행하면 됨.
mecab=Mecab()

!wget https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
!unzip v0.9.2.zip
%cd fastText-0.9.2
!make

# 형태소 분석 + 전처리

In [ ]:
train_test = pd.concat([train,test],axis=0)
train_test.reset_index(drop=True, inplace=True)

In [ ]:
X_data = train_test['content']

In [ ]:
%time

list1 = []
    
for text in X_data:
    token = mecab.morphs(text)
    list1.append(token)

# test csv 실제 예측 시작 준비
# list1은 train csv랑 test csv 합쳐서 토큰화 한 것을 리스트로 받은 것
test_token = list1[len(train):]

In [ ]:
# 트큰화 한 것을 리스트로 받으면서 불필요한 기호가 붙었음. 이것을 제거

for i in range(len(test_token)):
  test_token[i] = str(test_token[i]).strip('[]')
  test_token[i] = str(test_token[i]).replace(',','')
  test_token[i] = str(test_token[i]).replace("\'", "")

In [ ]:
# 모델에 바로 넣을 수 있게 전처리된 파일을 다음 위치에 저장

pd.DataFrame(test_token).to_csv('/content/mecab-ko-dic-2.1.1-20180720/fastText-0.9.2/fasttext_test.txt')

# 예측

In [ ]:
train_token = list1[:len(train)]

In [ ]:
for i in range(len(train_token)):
  train_token[i] = str(train_token[i]).strip('[]')
  train_token[i] = str(train_token[i]).replace(',','')
  train_token[i] = str(train_token[i]).replace("\'", "")

In [ ]:
for i in range(len(train)):
  train['info'][i] = '__label__'+str(train['info'][i])

In [ ]:
train_final = pd.concat([pd.DataFrame(train_token), train['info']],axis=1)

In [ ]:
train_final.columns = [0,1]

In [ ]:
# 파라미터 자동 최적화 위해 train과 validation 따로 나눠줌

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_final[0], train_final[1], test_size=0.2, random_state=42)

In [ ]:
# 파라미터 자동 최적화 위해 train과 validation 따로 나눠줌

train_final_t = pd.concat([X_train, y_train],axis=1)
train_final_v = pd.concat([X_test, y_test],axis=1)

In [ ]:
# 파라미터 자동 최적화 위해 train과 validation 따로 나눠줌

train_final_t.to_csv('/content/mecab-ko-dic-2.1.1-20180720/fastText-0.9.2/fasttext_t.txt', index=False, sep = '\t')
train_final_v.to_csv('/content/mecab-ko-dic-2.1.1-20180720/fastText-0.9.2/fasttext_v.txt', index=False, sep = '\t')

In [ ]:
train_final.to_csv('/content/mecab-ko-dic-2.1.1-20180720/fastText-0.9.2/fasttext.txt', index=False, sep = '\t')

In [ ]:
!pwd

In [ ]:
# 파라미터 자동 최적화

!./fasttext supervised -input ./fasttext_t.txt -output model -autotune-validation ./fasttext_v.txt

In [ ]:
# test csv 예측하여 result.txt로 저장
!./fasttext predict model.bin /content/mecab-ko-dic-2.1.1-20180720/fastText-0.9.2/fasttext_test.txt > result.txt

# 제출

In [ ]:
# 저장했던 result.txt를 불러와서 csv로 저장
result = pd.read_csv('./result.txt', delimiter='\t', header=None)

In [ ]:
result= result[0].str.replace(pat=r'__label__', repl= r'', regex=True)

In [ ]:
result= result[1:len(result)]
result.reset_index(inplace=True, drop=True)

In [ ]:
sub= pd.concat([test['id'], result], axis=1)
sub= sub.rename({'0':'info'}, axis=1)

In [ ]:
print(time.time() - start)

In [ ]:
sub.to_csv('/content/mnt/Shareddrives/Real_News/승후의 서재/sub.csv')